# 特生中心資料處理

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### 事先用人工查詢加入了「體型」維度，並整理了學名及中文名稱。

In [ ]:
x=pd.read_excel("data(清理乾淨).xlsx")
x.head()

In [ ]:
#x.groupby("科名").count()

In [ ]:
section=[]
for i in range(len(x["科名"])):
    a=list(str(x.loc[i,"科名"]))
    while " " in a:
        a.remove(" ")
        x.loc[i,"科名"]="".join(a)
    if x.loc[i,"科名"]=="Anura":
        x.loc[i,"科名"]="Anura無尾目"
    elif x.loc[i,"科名"]=="Aves":
        x.loc[i,"科名"]="Aves鳥綱"
    elif x.loc[i,"科名"]=="Charadriidae?科":
        x.loc[i,"科名"]="Charadriidae鴴科"
    elif x.loc[i,"科名"]=="Emberizidae?科":
        x.loc[i,"科名"]="Emberizidae鵐科"    
    elif x.loc[i,"科名"]=="Glareolidae燕?科":
        x.loc[i,"科名"]="Glareolidae燕鴴科"  
    elif x.loc[i,"科名"]=="Herpestidae?科":
        x.loc[i,"科名"]="Herpestidae獴科"  
    elif x.loc[i,"科名"]=="Podicipedidae??科":
        x.loc[i,"科名"]="Podicipedidae鸊鷉科"  
    elif x.loc[i,"科名"]=="Procellariidae?科":
        x.loc[i,"科名"]="Procellariidae鸌科"
    elif x.loc[i,"科名"]=="Procellariidae水薙鳥科":
        x.loc[i,"科名"]="Procellariidae鸌科"  
    elif x.loc[i,"科名"]=="Ocypodiae沙蟹科":
        x.loc[i,"科名"]="Ocypodidae沙蟹科"  
    elif x.loc[i,"科名"]=="Cettidae樹鶯科":
        x.loc[i,"科名"]="Cettiidae樹鶯科"  
    section.append(x.loc[i,"科名"])
x["科名"]=section

In [ ]:
len(set(x["科名"])) #125科+nan

### 從物種科別配適所屬的「綱」(一般所稱「分類」)

In [ ]:
animal=pd.read_excel("animal界.xlsx")
dict1={};dict2={}
for i in range(len(animal)):
    dict1.update({animal.loc[i,"科"]:animal.loc[i,"綱"]})
    dict2.update({animal.loc[i,"目"]:animal.loc[i,"綱"]})
x["綱"]=x["科名"].replace(dict1)
x["綱"]=x["綱"].replace(dict2)

In [ ]:
#x.groupby(["分類","綱"]).count()

In [ ]:
for i in range(len(x)):
    if x.loc[i,"綱"]=="Podicipedidae鷿鷈科":
        x.loc[i,"綱"]="Aves鳥綱"
    if x.loc[i,"綱"]=="Insectivora食蟲目":
        x.loc[i,"綱"]="Mammalia哺乳綱"
    if x.loc[i,"綱"]=="Scarabaeidae":
        x=x.drop(i,axis=0)

In [ ]:
#x.groupby(["分類","綱"]).count()

### 清理不一致的縣市名稱

In [ ]:
x=x.reset_index(drop=True)
for i in range(len(x)):
    if x.loc[i,"縣市"]=="台中市":
        x.loc[i,"縣市"]="臺中市"
    if x.loc[i,"縣市"]=="台北市":
        x.loc[i,"縣市"]="臺北市"
    if x.loc[i,"縣市"]=="台南市":
        x.loc[i,"縣市"]="臺南市"
    if x.loc[i,"縣市"]=="台東市":
        x.loc[i,"縣市"]="臺東市"
    if x.loc[i,"縣市"]=="台東縣":
        x.loc[i,"縣市"]="臺東縣"  

In [ ]:
x=x.drop("分類",axis=1)
x=x[['序號', '觀察日期', '縣市', '鄉鎮', '觀察地區', '經度', '緯度', '綱', '科名', '學名', '中名','體型(cm)']]

In [ ]:
#x.to_excel("data(清理乾淨).xlsx")

### 加入「節氣」維度

In [ ]:
y=pd.read_excel("路殺社資料集(含節氣).xlsx",sheet_name="路殺社資料集(含節氣)")
y.columns

In [ ]:
x["季"]=[np.nan]*len(x)
x["節"]=[np.nan]*len(x)
x["候"]=[np.nan]*len(x)
for i in range(len(x)):
    for j in range(len(y)):
        if x.loc[i,"序號"]==y.loc[j,"序號"]:
            x.loc[i,"季"]=y.loc[j,"季"]
            x.loc[i,"節"]=y.loc[j,"節"]
            x.loc[i,"候"]=y.loc[j,"候"]

In [ ]:
#set(x["候"]) #修誇跨謀

In [ ]:
#x.to_excel("data(清理乾淨_增加節氣).xlsx")

In [3]:
x=pd.read_excel("data(清理乾淨_增加節氣).xlsx")

### 加入溫度雨量日照等環境資訊

#### 先把環境資訊資料，加入地理位置。

In [4]:
from datetime import datetime
from shapely.geometry import Point

In [5]:
t=pd.read_excel("temp20190515.xlsx")
s=pd.read_excel("溫雨測站資料.xlsx")

In [6]:
s.head() #全台灣各測站的點位

,站號,站名,海拔高度(m),經度,緯度,城市,地址,資料起始日期,撤站日期,備註,原站號,新站號
0,466850,五分山雷達站,756.0,121.7812,25.0712,新北市,瑞芳區靜安路四段1巷1號,1988-07-01,NaN,本站只有雷達觀測資料。,NaN,NaN
1,466880,板橋,9.7,121.4420,24.9976,新北市,板橋區大觀路二段265巷62號,1972-03-01,NaN,原為探空站，自2002年開始進行氣象觀測。,NaN,NaN
2,466900,淡水,19.0,121.4489,25.1649,新北市,淡水區中正東路42巷6號,1942-01-01,NaN,NaN,NaN,NaN
3,466910,鞍部,825.8,121.5297,25.1826,臺北市,北投區陽明山竹子湖路111號,1937-01-01,NaN,NaN,NaN,NaN
4,466920,臺北,6.3,121.5149,25.0377,臺北市,中正區公園路64號,1896-01-01,NaN,2014至2015氣候年報之氣壓計海面上高度應為30.3m(2014/10/25搬遷，6.6...,NaN,NaN


In [7]:
t.head() #各大測站的環境資訊紀錄

,Unnamed: 0,測站,日期,均溫,低溫,高溫,相對溼度,降水量(mm),日照時數
0,0,嘉義,2013/10/31,23.137499,17.700001,28.700001,78.833336,0,8.5
1,1,嘉義,2013/10/30,23.050001,17.600000,29.500000,76.708336,0,9.6
2,2,嘉義,2013/10/29,22.054163,16.400000,28.100000,78.125000,0,9.2
3,3,嘉義,2013/10/28,21.387499,15.300000,28.200001,76.625000,0,9.3
4,4,嘉義,2013/10/27,20.529167,15.200000,27.299999,76.875000,0,8.7


In [8]:
t["點位"]=[""]*len(t)
for i in range(len(t)):
    ss=s[s["站名"]==t.loc[i,"測站"]].reset_index(drop=True)
    p=Point(ss.loc[0,"經度"],ss.loc[0,"緯度"])
    t.loc[i,"點位"]=p

In [9]:
t.head() #把點位配進去，並記錄成地理資料(Point)

,Unnamed: 0,測站,日期,均溫,低溫,高溫,相對溼度,降水量(mm),日照時數,點位
0,0,嘉義,2013/10/31,23.137499,17.700001,28.700001,78.833336,0,8.5,POINT (120.4329 23.4959)
1,1,嘉義,2013/10/30,23.050001,17.600000,29.500000,76.708336,0,9.6,POINT (120.4329 23.4959)
2,2,嘉義,2013/10/29,22.054163,16.400000,28.100000,78.125000,0,9.2,POINT (120.4329 23.4959)
3,3,嘉義,2013/10/28,21.387499,15.300000,28.200001,76.625000,0,9.3,POINT (120.4329 23.4959)
4,4,嘉義,2013/10/27,20.529167,15.200000,27.299999,76.875000,0,8.7,POINT (120.4329 23.4959)


In [13]:
import geopandas as gpd
t=t.rename(columns={"測站":"station","日期":"date","均溫":"avg_temp","低溫":"min_temp","高溫":"max_temp","相對溼度":"humidity","降水量(mm)":"precipitation","日照時數":"sun_hours","點位":"geometry",})
t=gpd.GeoDataFrame(t,geometry="geometry")
#t=t.drop("Unnamed: 0",axis=1)
t.to_file("tempfinal.shp",encoding="utf-8")

In [16]:
import geopandas as gpd
tt=gpd.read_file("tempfinal.shp",encoding="utf-8")

#### 在路殺資料中，加入離路殺位置最近的、當日的環境資料。

In [20]:
x["avg_temp"]=[np.nan]*len(x)
x["min_temp"]=[np.nan]*len(x)
x["max_temp"]=[np.nan]*len(x)
x["humidity"]=[np.nan]*len(x)
x["precipitation"]=[np.nan]*len(x)
x["sun_hours"]=[np.nan]*len(x)

In [21]:
nolatlonindex=[]
notemp=[]
for i in range(len(x)):
    if pd.isnull(x.loc[i,"經度"])==False:
        p=Point(x.loc[i,"經度"],x.loc[i,"緯度"])
        tt=t[ t["date"]==datetime.strftime(x.loc[i,"觀察日期"],"%Y/%m/%d")]
        tt=tt.reset_index(drop=True)
        dist=[]
        if len(tt)!=0:
            for j in range(len(tt)):
                d=p.distance(tt.loc[j,"geometry"])
                dist.append(d)
            jj=dist.index(np.min(dist))
            x.loc[i,"avg_temp"]=tt.loc[jj,"avg_temp"]
            x.loc[i,"min_temp"]=tt.loc[jj,"min_temp"]
            x.loc[i,"max_temp"]=tt.loc[jj,"max_temp"]
            x.loc[i,"humidity"]=tt.loc[jj,"humidity"]
            x.loc[i,"precipitation"]=tt.loc[jj,"precipitation"]
            x.loc[i,"sun_hours"]=tt.loc[jj,"sun_hours"]
        elif len(tt)==0:
            notemp.append(datetime.strftime(x.loc[i,"觀察日期"],"%Y/%m/%d"))
    elif pd.isnull(x.loc[i,"經度"])==True:
        nolatlonindex.append(i)

In [ ]:
#x.to_excel("data(清理乾淨_增加節氣_增加溫溼度).xlsx")

#### 針對無點位的路殺資料，以所在縣市將離最近的環境資料帶入。

In [25]:
#print(notemp)
print(len(notemp)) #2833

2833


In [26]:
#print( nolatlonindex)
print(len( nolatlonindex)) #1064

1064


In [73]:
xlost=x[pd.isnull(x["經度"])==True]
#xlost

In [75]:
print(set(xlost["縣市"]))

{nan, '南投縣', '金門縣', '臺北市', '雲林縣', '臺南市', '苗栗縣', '嘉義市', '嘉義縣', '花蓮縣', '彰化縣', '臺中市', '新北市', '新竹縣', '桃園縣', '宜蘭縣', '高雄市', '臺東縣', '新竹市', '屏東縣', '連江縣'}


In [76]:
print(set(t["station"]))

{'板橋', '恆春', '蘭嶼', '澎湖', '梧棲', '宜蘭', '臺北', '竹子湖', '嘉義', '成功', '基隆', '蘇澳', '玉山', '馬祖', '新屋', '臺中', '新竹', '臺東', '大武', '鞍部', '阿里山', '臺南', '日月潭', '花蓮', '高雄', '東吉島', '金門', '淡水', '彭佳嶼'}


In [68]:
for i in xlost.index:
    if x.loc[i,"縣市"]=="南投縣":
        tt=t[t["station"]=="日月潭"]
    if x.loc[i,"縣市"]=="金門縣":
        tt=t[t["station"]=="金門"]
    if x.loc[i,"縣市"]=="臺北市":
        tt=t[t["station"]=="臺北"]
    if x.loc[i,"縣市"]=="雲林縣":
        tt=t[t["station"]=="嘉義"]
    if x.loc[i,"縣市"]=="臺南市":
        tt=t[t["station"]=="臺南"]
    if x.loc[i,"縣市"]=="苗栗縣":
        tt=t[t["station"]=="新竹"]
    if x.loc[i,"縣市"]=="嘉義市":
        tt=t[t["station"]=="嘉義"]
    if x.loc[i,"縣市"]=="嘉義縣":
        tt=t[t["station"]=="嘉義"]
    if x.loc[i,"縣市"]=="花蓮縣":
        tt=t[t["station"]=="花蓮"]
    if x.loc[i,"縣市"]=="臺中市":
        tt=t[t["station"]=="臺中"]
    if x.loc[i,"縣市"]=="新北市":
        tt=t[t["station"]=="板橋"]
    if x.loc[i,"縣市"]=="新竹縣":
        tt=t[t["station"]=="新竹"]
    if x.loc[i,"縣市"]=="桃園縣":
        tt=t[t["station"]=="新屋"]
    if x.loc[i,"縣市"]=="宜蘭縣":
        tt=t[t["station"]=="宜蘭"]
    if x.loc[i,"縣市"]=="高雄市":
        tt=t[t["station"]=="高雄"]
    if x.loc[i,"縣市"]=="臺東縣":
        tt=t[t["station"]=="臺東"]
    if x.loc[i,"縣市"]=="新竹市":
        tt=t[t["station"]=="新竹"]
    if x.loc[i,"縣市"]=="屏東縣":
        tt=t[t["station"]=="恆春"]
    if x.loc[i,"縣市"]=="連江縣":
        tt=t[t["station"]=="馬祖"]
    tt=tt[ tt["date"]==datetime.strftime(x.loc[i,"觀察日期"],"%Y/%m/%d")]
    tt=tt.reset_index(drop=True)
    if len(tt)!=0:
        x.loc[i,"avg_temp"]=tt.loc[0,"avg_temp"]
        x.loc[i,"min_temp"]=tt.loc[0,"min_temp"]
        x.loc[i,"max_temp"]=tt.loc[0,"max_temp"]
        x.loc[i,"humidity"]=tt.loc[0,"humidity"]
        x.loc[i,"precipitation"]=tt.loc[0,"precipitation"]
        x.loc[i,"sun_hours"]=tt.loc[0,"sun_hours"]    
    tt=tt[ tt["date"]==datetime.strftime(x.loc[i,"觀察日期"],"%Y/%m/%d")]
    tt=tt.reset_index(drop=True)
    if len(tt)!=0:
        x.loc[i,"avg_temp"]=tt.loc[0,"avg_temp"]
        x.loc[i,"min_temp"]=tt.loc[0,"min_temp"]
        x.loc[i,"max_temp"]=tt.loc[0,"max_temp"]
        x.loc[i,"humidity"]=tt.loc[0,"humidity"]
        x.loc[i,"precipitation"]=tt.loc[0,"precipitation"]
        x.loc[i,"sun_hours"]=tt.loc[0,"sun_hours"]    

In [74]:
#x.to_excel("data(清理乾淨_增加節氣_增加溫溼度).xlsx")

### 配入道路資訊

In [21]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
from pylab import mpl
import json
import codecs
from folium.plugins import HeatMap
from shapely.geometry import LineString, Polygon, Point
%matplotlib inline

In [22]:
roaddata=gpd.read_file("roaddata.shp",encoding="utf-8")

In [96]:
x.head()

,Unnamed: 0,序號,觀察日期,縣市,鄉鎮,觀察地區,經度,緯度,科名,學名,...,min_temp,max_temp,humidity,precipitation,sun_hours,RdCode,Town,Village,Road_Name,distance
0,0,1,2015-12-31,臺中市,大肚區,NaN,120.572204,24.155384,Columbidae鳩鴿科,Streptopelia chinensis chinensis,...,15.3,19.200001,71.041667,0.0,9.3,NaN,,,,NaN
1,1,8225,2014-12-23,臺東縣,蘭嶼鄉,椰油村288號,121.509903,22.056647,Strigidae鴟鴞科,Otus elegans botelensis,...,15.4,18.299999,73.708333,0.0,0.5,NaN,,,,NaN
2,2,3,2015-12-31,臺北市,內湖區,NaN,121.572876,25.101981,Talpidae鼴鼠科,Mogera insularis insularis,...,10.4,11.400000,89.041667,1.0,0.0,NaN,,,,NaN
3,3,4,2015-12-31,臺北市,北投區,NaN,121.484387,25.153514,Colubridae黃頷蛇科,Oreocryptophis porphyraceus kawakamii,...,15.3,17.500000,67.750000,0.0,0.0,NaN,,,,NaN
4,4,8259,2014-12-17,臺東縣,鹿野鄉,永樂路,121.126495,22.945644,Strigidae鴟鴞科,Otus,...,13.4,16.299999,62.166667,0.0,0.0,NaN,,,,NaN


In [84]:
roaddata.head()

,CODE,ROADNUM,ROADALIAS,COUNTY,TOWN,VILLAGE,ROAD_NAME,WIDTH,geometry
0,Rd0,None,None,南投縣,國姓鄉,國姓村,None,19.022789,"LINESTRING (120.8542707 24.0353081, 120.854340..."
1,Rd1,None,None,南投縣,鹿谷鄉,和雅村,None,19.022789,"LINESTRING (120.7661941 23.7127226, 120.766160..."
2,Rd2,None,None,南投縣,鹿谷鄉,和雅村,None,19.022789,"LINESTRING (120.7743284 23.716407, 120.7744386..."
3,Rd3,None,None,南投縣,鹿谷鄉,和雅村,None,19.022789,"LINESTRING (120.8031121 23.6989364, 120.802910..."
4,Rd4,None,None,南投縣,鹿谷鄉,和雅村,None,19.022789,"LINESTRING (120.7759961 23.6903228, 120.775816..."


#### 路中心往外擴增一倍寬

In [87]:
x["RdCode"]=[np.nan]*len(x)
x["Town"]=[""]*len(x)
x["Village"]=[""]*len(x)
x["Road_Name"]=[""]*len(x)
x["distance"]=[np.nan]*len(x)

In [127]:
for i in range(len(x)):
    lon,lat=x.loc[i,"經度"],x.loc[i,"緯度"]
    point=Point(lon,lat)
    city=x.loc[i,"縣市"]
    print(i,city,point)
    r=roaddata[roaddata["COUNTY"]==city]
    r=r.reset_index(drop=True)    
    
    for j in range(len(r)):
        line=r.loc[j,"geometry"]
        width=r.loc[j,"WIDTH"] #往外擴增一倍寬都算
        length=line.distance(point)*111000
        if length < width:
            if pd.isnull(x.loc[i,"distance"])==True:
                x.loc[i,"RdCode"]=r.loc[j,"CODE"]
                x.loc[i,"Town"]=r.loc[j,"TOWN"]
                x.loc[i,"Village"]=r.loc[j,"VILLAGE"]
                x.loc[i,"Road_Name"]=r.loc[j,"ROAD_NAME"]
                x.loc[i,"distance"]=length
            if pd.isnull(x.loc[i,"distance"])==False and length<x.loc[i,"distance"]:
                x.loc[i,"RdCode"]=r.loc[j,"CODE"]
                x.loc[i,"Town"]=r.loc[j,"TOWN"]
                x.loc[i,"Village"]=r.loc[j,"VILLAGE"]
                x.loc[i,"Road_Name"]=r.loc[j,"ROAD_NAME"]
                x.loc[i,"distance"]=length

0 臺中市 POINT (120.572204 24.155384)
1 臺東縣 POINT (121.509903 22.056647)
2 臺北市 POINT (121.572876 25.101981)
3 臺北市 POINT (121.484387 25.153514)
4 臺東縣 POINT (121.126495 22.945644)
5 嘉義市 POINT (120.438899 23.479248)
6 桃園市 POINT (121.318026 25.046119)
7 臺東縣 POINT (121.103622 22.88175)
8 臺中市 POINT (121.310131 24.352805)
9 高雄市 POINT (120.212119 22.880803)
10 南投縣 POINT (120.942604 23.792778)
11 苗栗縣 POINT (120.783183 24.409833)
12 臺南市 POINT (120.33346 23.25076)
13 臺南市 POINT (120.33346 23.25076)
14 臺南市 POINT (120.306953 23.204602)
15 臺南市 POINT (120.306953 23.204602)
16 臺南市 POINT (120.306953 23.204602)
17 臺南市 POINT (120.322522 23.264223)
18 臺東縣 POINT (121.176277 23.041824)
19 臺東縣 POINT (121.173721 23.040798)
20 臺東縣 POINT (121.176277 23.041824)
21 桃園市 POINT (121.345197 25.058736)
22 高雄市 POINT (120.635964 23.159833)
23 臺南市 POINT (120.333608 23.252716)
24 臺南市 POINT (120.333608 23.252716)
25 臺南市 POINT (120.333608 23.252716)
26 臺南市 POINT (120.33284 23.250632)
27 臺東縣 POINT (121.15022 22.8347)
28 臺東縣 POIN

228 臺南市 POINT (120.466475 23.267295)
229 新北市 POINT (121.46342 25.215998)
230 臺東縣 POINT (120.893822 22.357867)
231 嘉義縣 POINT (120.652926 23.423898)
232 嘉義縣 POINT (120.675821 23.47978)
233 嘉義縣 POINT (120.677058 23.482235)
234 嘉義縣 POINT (120.679387 23.484636)
235 宜蘭縣 POINT (121.748922 24.77326)
236 彰化縣 POINT (120.418461 24.046733)
237 新北市 POINT (121.561183 24.902893)
238 新北市 POINT (121.45818 25.141893)
239 金門縣 POINT (118.432026 24.472954)
240 嘉義縣 POINT (120.656389 23.420555)
241 臺中市 POINT (120.581389 24.204167)
242 臺北市 POINT (121.46459 25.116025)
243 臺北市 POINT (121.467302 25.115174)
244 臺南市 POINT (120.379987 23.249249)
245 臺南市 POINT (120.35372 23.198011)
246 臺南市 POINT (120.380611 23.251356)
247 宜蘭縣 POINT (121.718033 24.767188)
248 彰化縣 POINT (120.389389 24.028583)
249 桃園市 POINT (121.339099 25.06096)
250 苗栗縣 POINT (120.735939 24.406549)
251 臺南市 POINT (120.362983 23.208675)
252 桃園市 POINT (121.303909 25.053891)
253 桃園市 POINT (121.340144 25.065313)
254 桃園市 POINT (121.356851 25.056756)
255 高雄市 

452 桃園市 POINT (121.383782 24.678192)
453 桃園市 POINT (121.383782 24.678192)
454 花蓮縣 POINT (121.653559 24.124752)
455 屏東縣 POINT (120.62052 22.55013)
456 金門縣 POINT (118.229004 24.432418)
457 雲林縣 POINT (120.539558 23.653205)
458 雲林縣 POINT (120.541957 23.651615)
459 雲林縣 POINT (120.516947 23.644485)
460 雲林縣 POINT (120.521927 23.641765)
461 雲林縣 POINT (120.544519 23.679698)
462 雲林縣 POINT (120.508442 23.656015)
463 雲林縣 POINT (120.502812 23.657278)
464 雲林縣 POINT (120.502812 23.652938)
465 雲林縣 POINT (120.52772 23.653155)
466 雲林縣 POINT (120.502308 23.657737)
467 南投縣 POINT (120.735517 23.763734)
468 宜蘭縣 POINT (121.579833 24.652737)
469 臺北市 POINT (121.569613 25.058935)
470 臺南市 POINT (120.13055 23.01806)
471 嘉義市 POINT (120.485596 23.467031)
472 南投縣 POINT (120.69535 23.85679)
473 嘉義縣 POINT (120.166837 23.309967)
474 嘉義縣 POINT (120.155107 23.501529)
475 高雄市 POINT (120.709028 23.036464)
476 臺中市 POINT (120.965414 24.145714)
477 臺東縣 POINT (121.27736 22.942095)
478 金門縣 POINT (118.319553 24.424531)
479 金門縣 P

676 臺東縣 POINT (121.055896 23.15822)
677 臺東縣 POINT (121.110012 23.138915)
678 臺東縣 POINT (121.134361 23.132694)
679 南投縣 POINT (120.847969 24.000082)
680 嘉義縣 POINT (120.409859 23.584722)
681 宜蘭縣 POINT (121.422883 24.48355)
682 新北市 POINT (121.523553 25.181438)
683 新竹縣 POINT (121.081185 24.787955)
684 南投縣 POINT (120.73363 23.83669)
685 桃園市 POINT (121.327701 25.059312)
686 雲林縣 POINT (120.571999 23.601814)
687 雲林縣 POINT (120.572575 23.601862)
688 雲林縣 POINT (120.524297 23.640645)
689 臺中市 POINT (120.71231 24.16741)
690 臺中市 POINT (120.638799 24.135246)
691 臺北市 POINT (121.5631 25.106468)
692 臺北市 POINT (121.55633 25.081576)
693 臺南市 POINT (120.286957 23.244158)
694 臺南市 POINT (120.361868 23.242253)
695 屏東縣 POINT (120.755315 22.409921)
696 屏東縣 POINT (120.725552 22.413463)
697 新北市 POINT (121.403126 24.858714)
698 新北市 POINT (121.556666 25.276521)
699 桃園市 POINT (121.4233 24.64983)
700 花蓮縣 POINT (121.400127 23.717817)
701 金門縣 POINT (118.425495 24.433066)
702 新竹縣 POINT (121.022033 24.730553)
703 雲林縣 POINT

904 嘉義縣 POINT (120.325111 23.567347)
905 嘉義縣 POINT (120.332001 23.571396)
906 嘉義縣 POINT (120.5004 23.560733)
907 嘉義縣 POINT (120.417148 23.562235)
908 嘉義縣 POINT (120.507117 23.557667)
909 嘉義縣 POINT (120.518783 23.552717)
910 宜蘭縣 POINT (121.72941 24.640606)
911 nan POINT (121.553 24.63505)
912 宜蘭縣 POINT (121.453908 24.642289)
913 新北市 POINT (121.602258 25.149874)
914 新竹縣 POINT (121.044289 24.749964)
915 新竹縣 POINT (121.26617 24.59167)
916 桃園市 POINT (121.327262 25.059179)
917 花蓮縣 POINT (121.273506 23.204815)
918 嘉義市 POINT (120.478833 23.479172)
919 雲林縣 POINT (120.348061 23.595911)
920 雲林縣 POINT (120.348419 23.595957)
921 雲林縣 POINT (120.34301 23.59572)
922 雲林縣 POINT (120.312279 23.577126)
923 雲林縣 POINT (120.337334 23.597815)
924 雲林縣 POINT (120.331215 23.595137)
925 嘉義市 POINT (120.471925 23.474983)
926 雲林縣 POINT (120.312279 23.577126)
927 嘉義縣 POINT (120.571458 23.374006)
928 新竹縣 POINT (121.006963 24.69768)
929 雲林縣 POINT (120.583687 23.594798)
930 雲林縣 POINT (120.577185 23.595309)
931 高雄市 POINT

...

19757 嘉義縣 POINT (120.524514 23.4717)
19758 嘉義縣 POINT (120.523631 23.465458)
19759 嘉義縣 POINT (120.599415 23.439423)
19760 屏東縣 POINT (nan nan)
19761 新北市 POINT (121.433724 25.145137)
19762 新北市 POINT (121.434217 25.144504)
19763 新北市 POINT (nan nan)
19764 新竹縣 POINT (121.310448 24.685507)
19765 桃園縣 POINT (121.308867 24.825)
19766 桃園縣 POINT (nan nan)
19767 桃園縣 POINT (nan nan)
19768 桃園縣 POINT (nan nan)
19769 桃園縣 POINT (nan nan)
19770 桃園縣 POINT (nan nan)
19771 桃園縣 POINT (nan nan)
19772 屏東縣 POINT (120.852654 21.903319)
19773 高雄市 POINT (nan nan)
19774 臺中市 POINT (121.30772 24.38063)
19775 臺中市 POINT (121.309444 24.359444)
19776 新竹縣 POINT (121.11556 24.503761)
19777 臺北市 POINT (121.560073 25.108645)
19778 臺北市 POINT (121.561403 25.10744)
19779 臺東縣 POINT (121.231395 23.121106)
19780 新北市 POINT (nan nan)
19781 新竹縣 POINT (121.070371 24.698881)
19782 新竹縣 POINT (121.072018 24.69961)
19783 新竹縣 POINT (121.072018 24.69961)
19784 桃園縣 POINT (121.42142 24.64882)
19785 桃園縣 POINT (121.4044 24.67384)
19786 桃園縣 POINT

20003 臺北市 POINT (121.5585 25.181333)
20004 南投縣 POINT (121.17573 24.09281)
20005 臺北市 POINT (121.540767 25.166883)
20006 臺北市 POINT (121.544167 25.164433)
20007 臺北市 POINT (121.544584 25.035525)
20008 臺北市 POINT (121.561661 25.105731)
20009 臺北市 POINT (121.554623 25.099124)
20010 臺北市 POINT (121.556433 25.180933)
20011 臺北市 POINT (nan nan)
20012 臺南市 POINT (120.20869 22.987138)
20013 臺南市 POINT (120.522255 23.654481)
20014 臺東縣 POINT (nan nan)
20015 臺東縣 POINT (nan nan)
20016 彰化縣 POINT (nan nan)
20017 桃園市 POINT (121.18489 24.945949)
20018 新北市 POINT (nan nan)
20019 花蓮縣 POINT (121.409754 23.701195)
20020 花蓮縣 POINT (121.469231 23.756658)
20021 雲林縣 POINT (120.519008 23.64333)
20022 雲林縣 POINT (120.522255 23.654481)
20023 雲林縣 POINT (120.53121 23.652158)
20024 雲林縣 POINT (120.534528 23.651103)
20025 臺南市 POINT (nan nan)
20026 臺南市 POINT (nan nan)
20027 嘉義縣 POINT (nan nan)
20028 臺南市 POINT (120.370132 23.2387)
20029 新北市 POINT (121.430977 25.144489)
20030 雲林縣 POINT (120.535181 23.696124)
20031 桃園縣 POINT (121.3

20245 臺北市 POINT (121.59392 25.18163)
20246 臺北市 POINT (121.59314 25.13254)
20247 臺北市 POINT (121.59314 25.13253)
20248 嘉義縣 POINT (120.604147 23.46303)
20249 嘉義縣 POINT (120.586541 23.458827)
20250 宜蘭縣 POINT (121.711948 24.717579)
20251 新北市 POINT (121.764012 24.871218)
20252 新北市 POINT (121.70278 25.00813)
20253 桃園縣 POINT (121.208642 25.068456)
20254 桃園縣 POINT (nan nan)
20255 桃園市 POINT (121.342222 25.065591)
20256 苗栗縣 POINT (120.840808 24.368389)
20257 高雄市 POINT (nan nan)
20258 nan POINT (120.977833 24.238474)
20259 南投縣 POINT (120.725412 23.836504)
20260 嘉義縣 POINT (nan nan)
20261 宜蘭縣 POINT (121.79814 24.46386)
20262 宜蘭縣 POINT (121.536852 24.494633)
20263 宜蘭縣 POINT (121.7734 24.46572)
20264 桃園市 POINT (121.335467 25.062838)
20265 nan POINT (121.532886 24.919444)
20266 南投縣 POINT (120.764075 23.818295)
20267 臺北市 POINT (121.573082 25.09071)
20268 臺北市 POINT (nan nan)
20269 臺南市 POINT (120.34872 23.188101)
20270 臺南市 POINT (120.321238 23.38244)
20271 臺東縣 POINT (121.50254 22.67426)
20272 臺東縣 POINT (1

20471 臺南市 POINT (120.068786 23.084499)
20472 臺南市 POINT (120.403306 23.286425)
20473 臺南市 POINT (120.388591 23.294619)
20474 臺南市 POINT (120.723653 23.211167)
20475 新北市 POINT (121.513022 25.229572)
20476 臺南市 POINT (120.126443 23.034373)
20477 臺南市 POINT (120.468221 23.275323)
20478 臺南市 POINT (120.219901 23.267506)
20479 臺南市 POINT (120.219901 23.267506)
20480 臺東縣 POINT (121.165315 22.853948)
20481 嘉義縣 POINT (nan nan)
20482 嘉義縣 POINT (nan nan)
20483 嘉義縣 POINT (nan nan)
20484 嘉義縣 POINT (nan nan)
20485 雲林縣 POINT (120.50856 23.652001)
20486 嘉義縣 POINT (nan nan)
20487 嘉義縣 POINT (nan nan)
20488 嘉義縣 POINT (nan nan)
20489 嘉義縣 POINT (nan nan)
20490 嘉義縣 POINT (nan nan)
20491 嘉義縣 POINT (nan nan)
20492 嘉義縣 POINT (nan nan)
20493 屏東縣 POINT (120.152552 22.998605)
20494 新竹縣 POINT (nan nan)
20495 南投縣 POINT (nan nan)
20496 南投縣 POINT (120.881993 23.621256)
20497 臺南市 POINT (120.310539 23.184106)
20498 臺南市 POINT (120.39291 23.155745)
20499 臺南市 POINT (120.35214 23.125033)
20500 臺南市 POINT (120.429825 23.145552)
20

20696 臺南市 POINT (120.533741 23.651348)
20697 臺南市 POINT (120.801287 23.827559)
20698 臺南市 POINT (120.486525 23.672446)
20699 南投縣 POINT (121.16782 24.06982)
20700 南投縣 POINT (121.16782 24.06982)
20701 臺北市 POINT (121.506111 25.122222)
20702 臺南市 POINT (120.511098 23.649803)
20703 臺南市 POINT (120.492363 23.67089)
20704 屏東縣 POINT (120.796548 22.221628)
20705 臺南市 POINT (120.507521 23.659853)
20706 新竹縣 POINT (121.769604 24.437755)
20707 臺東縣 POINT (121.162008 22.842843)
20708 桃園縣 POINT (121.533501 25.064429)
20709 臺北市 POINT (121.550074 25.086376)
20710 臺南市 POINT (120.472375 23.675052)
20711 彰化縣 POINT (120.931664 24.774656)
20712 臺南市 POINT (120.377712 23.287317)
20713 臺南市 POINT (120.81305 23.949473)
20714 臺南市 POINT (120.398183 23.290165)
20715 臺南市 POINT (120.327751 23.291483)
20716 臺南市 POINT (120.352607 23.287199)
20717 新竹縣 POINT (121.739035 24.793724)
20718 新竹縣 POINT (121.742409 24.797561)
20719 新竹縣 POINT (121.738021 24.797727)
20720 新竹縣 POINT (121.739481 24.792798)
20721 臺南市 POINT (120.113298 23.

In [135]:
len(x[pd.isnull(x["RdCode"])==False])

3103

In [130]:
#x.to_excel("data(清理乾淨_增加節氣_增加溫溼度_增加道路資料A).xlsx")

#### 路中心往外擴增二倍寬

In [132]:
x["RdCode2"]=[np.nan]*len(x)
x["Town2"]=[""]*len(x)
x["Village2"]=[""]*len(x)
x["Road_Name2"]=[""]*len(x)
x["distance2"]=[np.nan]*len(x)

In [133]:
for i in range(len(x)):
    lon,lat=x.loc[i,"經度"],x.loc[i,"緯度"]
    point=Point(lon,lat)
    city=x.loc[i,"縣市"]
    print(i,city,point)
    r=roaddata[roaddata["COUNTY"]==city]
    r=r.reset_index(drop=True)    
    
    for j in range(len(r)):
        line=r.loc[j,"geometry"]
        width=r.loc[j,"WIDTH"]*2 #往外擴增二倍寬都算
        length=line.distance(point)*111000
        if length < width:
            if pd.isnull(x.loc[i,"distance2"])==True:
                x.loc[i,"RdCode2"]=r.loc[j,"CODE"]
                x.loc[i,"Town2"]=r.loc[j,"TOWN"]
                x.loc[i,"Village2"]=r.loc[j,"VILLAGE"]
                x.loc[i,"Road_Name2"]=r.loc[j,"ROAD_NAME"]
                x.loc[i,"distance2"]=length
            if pd.isnull(x.loc[i,"distance2"])==False and length<x.loc[i,"distance2"]:
                x.loc[i,"RdCode2"]=r.loc[j,"CODE"]
                x.loc[i,"Town2"]=r.loc[j,"TOWN"]
                x.loc[i,"Village2"]=r.loc[j,"VILLAGE"]
                x.loc[i,"Road_Name2"]=r.loc[j,"ROAD_NAME"]
                x.loc[i,"distance2"]=length

0 臺中市 POINT (120.572204 24.155384)
1 臺東縣 POINT (121.509903 22.056647)
2 臺北市 POINT (121.572876 25.101981)
3 臺北市 POINT (121.484387 25.153514)
4 臺東縣 POINT (121.126495 22.945644)
5 嘉義市 POINT (120.438899 23.479248)
6 桃園市 POINT (121.318026 25.046119)
7 臺東縣 POINT (121.103622 22.88175)
8 臺中市 POINT (121.310131 24.352805)
9 高雄市 POINT (120.212119 22.880803)
10 南投縣 POINT (120.942604 23.792778)
11 苗栗縣 POINT (120.783183 24.409833)
12 臺南市 POINT (120.33346 23.25076)
13 臺南市 POINT (120.33346 23.25076)
14 臺南市 POINT (120.306953 23.204602)
15 臺南市 POINT (120.306953 23.204602)
16 臺南市 POINT (120.306953 23.204602)
17 臺南市 POINT (120.322522 23.264223)
18 臺東縣 POINT (121.176277 23.041824)
19 臺東縣 POINT (121.173721 23.040798)
20 臺東縣 POINT (121.176277 23.041824)
21 桃園市 POINT (121.345197 25.058736)
22 高雄市 POINT (120.635964 23.159833)
23 臺南市 POINT (120.333608 23.252716)
24 臺南市 POINT (120.333608 23.252716)
25 臺南市 POINT (120.333608 23.252716)
26 臺南市 POINT (120.33284 23.250632)
27 臺東縣 POINT (121.15022 22.8347)
28 臺東縣 POIN

228 臺南市 POINT (120.466475 23.267295)
229 新北市 POINT (121.46342 25.215998)
230 臺東縣 POINT (120.893822 22.357867)
231 嘉義縣 POINT (120.652926 23.423898)
232 嘉義縣 POINT (120.675821 23.47978)
233 嘉義縣 POINT (120.677058 23.482235)
234 嘉義縣 POINT (120.679387 23.484636)
235 宜蘭縣 POINT (121.748922 24.77326)
236 彰化縣 POINT (120.418461 24.046733)
237 新北市 POINT (121.561183 24.902893)
238 新北市 POINT (121.45818 25.141893)
239 金門縣 POINT (118.432026 24.472954)
240 嘉義縣 POINT (120.656389 23.420555)
241 臺中市 POINT (120.581389 24.204167)
242 臺北市 POINT (121.46459 25.116025)
243 臺北市 POINT (121.467302 25.115174)
244 臺南市 POINT (120.379987 23.249249)
245 臺南市 POINT (120.35372 23.198011)
246 臺南市 POINT (120.380611 23.251356)
247 宜蘭縣 POINT (121.718033 24.767188)
248 彰化縣 POINT (120.389389 24.028583)
249 桃園市 POINT (121.339099 25.06096)
250 苗栗縣 POINT (120.735939 24.406549)
251 臺南市 POINT (120.362983 23.208675)
252 桃園市 POINT (121.303909 25.053891)
253 桃園市 POINT (121.340144 25.065313)
254 桃園市 POINT (121.356851 25.056756)
255 高雄市 

452 桃園市 POINT (121.383782 24.678192)
453 桃園市 POINT (121.383782 24.678192)
454 花蓮縣 POINT (121.653559 24.124752)
455 屏東縣 POINT (120.62052 22.55013)
456 金門縣 POINT (118.229004 24.432418)
457 雲林縣 POINT (120.539558 23.653205)
458 雲林縣 POINT (120.541957 23.651615)
459 雲林縣 POINT (120.516947 23.644485)
460 雲林縣 POINT (120.521927 23.641765)
461 雲林縣 POINT (120.544519 23.679698)
462 雲林縣 POINT (120.508442 23.656015)
463 雲林縣 POINT (120.502812 23.657278)
464 雲林縣 POINT (120.502812 23.652938)
465 雲林縣 POINT (120.52772 23.653155)
466 雲林縣 POINT (120.502308 23.657737)
467 南投縣 POINT (120.735517 23.763734)
468 宜蘭縣 POINT (121.579833 24.652737)
469 臺北市 POINT (121.569613 25.058935)
470 臺南市 POINT (120.13055 23.01806)
471 嘉義市 POINT (120.485596 23.467031)
472 南投縣 POINT (120.69535 23.85679)
473 嘉義縣 POINT (120.166837 23.309967)
474 嘉義縣 POINT (120.155107 23.501529)
475 高雄市 POINT (120.709028 23.036464)
476 臺中市 POINT (120.965414 24.145714)
477 臺東縣 POINT (121.27736 22.942095)
478 金門縣 POINT (118.319553 24.424531)
479 金門縣 P

676 臺東縣 POINT (121.055896 23.15822)
677 臺東縣 POINT (121.110012 23.138915)
678 臺東縣 POINT (121.134361 23.132694)
679 南投縣 POINT (120.847969 24.000082)
680 嘉義縣 POINT (120.409859 23.584722)
681 宜蘭縣 POINT (121.422883 24.48355)
682 新北市 POINT (121.523553 25.181438)
683 新竹縣 POINT (121.081185 24.787955)
684 南投縣 POINT (120.73363 23.83669)
685 桃園市 POINT (121.327701 25.059312)
686 雲林縣 POINT (120.571999 23.601814)
687 雲林縣 POINT (120.572575 23.601862)
688 雲林縣 POINT (120.524297 23.640645)
689 臺中市 POINT (120.71231 24.16741)
690 臺中市 POINT (120.638799 24.135246)
691 臺北市 POINT (121.5631 25.106468)
692 臺北市 POINT (121.55633 25.081576)
693 臺南市 POINT (120.286957 23.244158)
694 臺南市 POINT (120.361868 23.242253)
695 屏東縣 POINT (120.755315 22.409921)
696 屏東縣 POINT (120.725552 22.413463)
697 新北市 POINT (121.403126 24.858714)
698 新北市 POINT (121.556666 25.276521)
699 桃園市 POINT (121.4233 24.64983)
700 花蓮縣 POINT (121.400127 23.717817)
701 金門縣 POINT (118.425495 24.433066)
702 新竹縣 POINT (121.022033 24.730553)
703 雲林縣 POINT

904 嘉義縣 POINT (120.325111 23.567347)
905 嘉義縣 POINT (120.332001 23.571396)
906 嘉義縣 POINT (120.5004 23.560733)
907 嘉義縣 POINT (120.417148 23.562235)
908 嘉義縣 POINT (120.507117 23.557667)
909 嘉義縣 POINT (120.518783 23.552717)
910 宜蘭縣 POINT (121.72941 24.640606)
911 nan POINT (121.553 24.63505)
912 宜蘭縣 POINT (121.453908 24.642289)
913 新北市 POINT (121.602258 25.149874)
914 新竹縣 POINT (121.044289 24.749964)
915 新竹縣 POINT (121.26617 24.59167)
916 桃園市 POINT (121.327262 25.059179)
917 花蓮縣 POINT (121.273506 23.204815)
918 嘉義市 POINT (120.478833 23.479172)
919 雲林縣 POINT (120.348061 23.595911)
920 雲林縣 POINT (120.348419 23.595957)
921 雲林縣 POINT (120.34301 23.59572)
922 雲林縣 POINT (120.312279 23.577126)
923 雲林縣 POINT (120.337334 23.597815)
924 雲林縣 POINT (120.331215 23.595137)
925 嘉義市 POINT (120.471925 23.474983)
926 雲林縣 POINT (120.312279 23.577126)
927 嘉義縣 POINT (120.571458 23.374006)
928 新竹縣 POINT (121.006963 24.69768)
929 雲林縣 POINT (120.583687 23.594798)
930 雲林縣 POINT (120.577185 23.595309)
931 高雄市 POINT

...

19751 臺北市 POINT (nan nan)
19752 臺南市 POINT (120.489631 23.459653)
19753 臺東縣 POINT (121.079271 23.143349)
19754 臺東縣 POINT (nan nan)
19755 嘉義市 POINT (120.489631 23.459653)
19756 嘉義縣 POINT (120.153185 23.503104)
19757 嘉義縣 POINT (120.524514 23.4717)
19758 嘉義縣 POINT (120.523631 23.465458)
19759 嘉義縣 POINT (120.599415 23.439423)
19760 屏東縣 POINT (nan nan)
19761 新北市 POINT (121.433724 25.145137)
19762 新北市 POINT (121.434217 25.144504)
19763 新北市 POINT (nan nan)
19764 新竹縣 POINT (121.310448 24.685507)
19765 桃園縣 POINT (121.308867 24.825)
19766 桃園縣 POINT (nan nan)
19767 桃園縣 POINT (nan nan)
19768 桃園縣 POINT (nan nan)
19769 桃園縣 POINT (nan nan)
19770 桃園縣 POINT (nan nan)
19771 桃園縣 POINT (nan nan)
19772 屏東縣 POINT (120.852654 21.903319)
19773 高雄市 POINT (nan nan)
19774 臺中市 POINT (121.30772 24.38063)
19775 臺中市 POINT (121.309444 24.359444)
19776 新竹縣 POINT (121.11556 24.503761)
19777 臺北市 POINT (121.560073 25.108645)
19778 臺北市 POINT (121.561403 25.10744)
19779 臺東縣 POINT (121.231395 23.121106)
19780 新北市 POINT (nan 

19993 雲林縣 POINT (120.510143 23.650565)
19994 雲林縣 POINT (120.523715 23.654353)
19995 雲林縣 POINT (120.531191 23.652136)
19996 nan POINT (nan nan)
19997 nan POINT (120.62061 23.96288)
19998 nan POINT (121.329681 23.430622)
19999 南投縣 POINT (121.08437 24.00258)
20000 南投縣 POINT (121.16802 24.06777)
20001 南投縣 POINT (nan nan)
20002 南投縣 POINT (121.20925 24.10897)
20003 臺北市 POINT (121.5585 25.181333)
20004 南投縣 POINT (121.17573 24.09281)
20005 臺北市 POINT (121.540767 25.166883)
20006 臺北市 POINT (121.544167 25.164433)
20007 臺北市 POINT (121.544584 25.035525)
20008 臺北市 POINT (121.561661 25.105731)
20009 臺北市 POINT (121.554623 25.099124)
20010 臺北市 POINT (121.556433 25.180933)
20011 臺北市 POINT (nan nan)
20012 臺南市 POINT (120.20869 22.987138)
20013 臺南市 POINT (120.522255 23.654481)
20014 臺東縣 POINT (nan nan)
20015 臺東縣 POINT (nan nan)
20016 彰化縣 POINT (nan nan)
20017 桃園市 POINT (121.18489 24.945949)
20018 新北市 POINT (nan nan)
20019 花蓮縣 POINT (121.409754 23.701195)
20020 花蓮縣 POINT (121.469231 23.756658)
20021 雲林縣 POI

20236 南投縣 POINT (120.911601 23.947133)
20237 南投縣 POINT (120.911601 23.947133)
20238 南投縣 POINT (120.911601 23.947133)
20239 南投縣 POINT (120.911601 23.947133)
20240 南投縣 POINT (120.911601 23.947133)
20241 南投縣 POINT (120.911601 23.947133)
20242 南投縣 POINT (120.911601 23.947133)
20243 南投縣 POINT (120.911601 23.947133)
20244 臺中市 POINT (nan nan)
20245 臺北市 POINT (121.59392 25.18163)
20246 臺北市 POINT (121.59314 25.13254)
20247 臺北市 POINT (121.59314 25.13253)
20248 嘉義縣 POINT (120.604147 23.46303)
20249 嘉義縣 POINT (120.586541 23.458827)
20250 宜蘭縣 POINT (121.711948 24.717579)
20251 新北市 POINT (121.764012 24.871218)
20252 新北市 POINT (121.70278 25.00813)
20253 桃園縣 POINT (121.208642 25.068456)
20254 桃園縣 POINT (nan nan)
20255 桃園市 POINT (121.342222 25.065591)
20256 苗栗縣 POINT (120.840808 24.368389)
20257 高雄市 POINT (nan nan)
20258 nan POINT (120.977833 24.238474)
20259 南投縣 POINT (120.725412 23.836504)
20260 嘉義縣 POINT (nan nan)
20261 宜蘭縣 POINT (121.79814 24.46386)
20262 宜蘭縣 POINT (121.536852 24.494633)
20263 宜蘭縣 

20462 臺南市 POINT (120.437976 23.468995)
20463 臺南市 POINT (120.176166 23.343994)
20464 臺南市 POINT (120.553918 23.627754)
20465 臺南市 POINT (120.172234 23.338059)
20466 臺南市 POINT (120.437976 23.468995)
20467 臺南市 POINT (120.437976 23.468995)
20468 臺南市 POINT (120.437976 23.468995)
20469 新竹縣 POINT (121.310121 24.347647)
20470 新竹縣 POINT (121.310121 24.347647)
20471 臺南市 POINT (120.068786 23.084499)
20472 臺南市 POINT (120.403306 23.286425)
20473 臺南市 POINT (120.388591 23.294619)
20474 臺南市 POINT (120.723653 23.211167)
20475 新北市 POINT (121.513022 25.229572)
20476 臺南市 POINT (120.126443 23.034373)
20477 臺南市 POINT (120.468221 23.275323)
20478 臺南市 POINT (120.219901 23.267506)
20479 臺南市 POINT (120.219901 23.267506)
20480 臺東縣 POINT (121.165315 22.853948)
20481 嘉義縣 POINT (nan nan)
20482 嘉義縣 POINT (nan nan)
20483 嘉義縣 POINT (nan nan)
20484 嘉義縣 POINT (nan nan)
20485 雲林縣 POINT (120.50856 23.652001)
20486 嘉義縣 POINT (nan nan)
20487 嘉義縣 POINT (nan nan)
20488 嘉義縣 POINT (nan nan)
20489 嘉義縣 POINT (nan nan)
20490 嘉義縣 POI

20687 桃園縣 POINT (121.368111 24.806089)
20688 桃園縣 POINT (nan nan)
20689 桃園縣 POINT (121.355911 24.714856)
20690 新竹縣 POINT (121.799924 24.443363)
20691 新北市 POINT (121.458753 25.153966)
20692 臺南市 POINT (120.101895 23.0899)
20693 臺南市 POINT (120.274705 22.996415)
20694 花蓮縣 POINT (121.208757 23.132901)
20695 臺南市 POINT (120.269698 23.033325)
20696 臺南市 POINT (120.533741 23.651348)
20697 臺南市 POINT (120.801287 23.827559)
20698 臺南市 POINT (120.486525 23.672446)
20699 南投縣 POINT (121.16782 24.06982)
20700 南投縣 POINT (121.16782 24.06982)
20701 臺北市 POINT (121.506111 25.122222)
20702 臺南市 POINT (120.511098 23.649803)
20703 臺南市 POINT (120.492363 23.67089)
20704 屏東縣 POINT (120.796548 22.221628)
20705 臺南市 POINT (120.507521 23.659853)
20706 新竹縣 POINT (121.769604 24.437755)
20707 臺東縣 POINT (121.162008 22.842843)
20708 桃園縣 POINT (121.533501 25.064429)
20709 臺北市 POINT (121.550074 25.086376)
20710 臺南市 POINT (120.472375 23.675052)
20711 彰化縣 POINT (120.931664 24.774656)
20712 臺南市 POINT (120.377712 23.287317)
20713 

In [136]:
len(x[pd.isnull(x["RdCode2"])==False])

4131

In [137]:
x.to_excel("data(清理乾淨_增加節氣_增加溫溼度_增加道路資料B).xlsx")

### 縣市名稱資料清理

In [42]:
import pandas as pd
x=pd.read_excel("data(清理乾淨_增加節氣_增加溫溼度_增加道路資料B).xlsx")

In [4]:
x["縣市"]=x["縣市"].replace({"桃園縣":"桃園市"})

In [7]:
x["臺灣分區"]=x["縣市"].replace({"臺北市":"北",
                             "新北市":"北",
                             "基隆市":"北",
                             "桃園市":"北",
                             "新竹市":"北",
                             "新竹縣":"北",
                             "宜蘭縣":"北",
                             "苗栗縣":"中",
                             "臺中市":"中",
                             "彰化縣":"中",
                             "南投縣":"中",
                             "雲林縣":"中",
                             "嘉義市":"南",
                             "嘉義縣":"南",
                             "臺南市":"南",
                             "高雄市":"南",
                             "屏東縣":"南",
                             "澎湖縣":"離島",
                             "花蓮縣":"東",
                             "臺東縣":"東",
                             "金門縣":"離島",
                             "連江縣":"離島"})

In [18]:
x["縣市(北中南東)"]=x["縣市"].replace({"臺北市":"02臺北市",
                             "新北市":"01新北市",
                             "基隆市":"00基隆市",
                             "桃園市":"03桃園市",
                             "新竹市":"05新竹市",
                             "新竹縣":"04新竹縣",
                             "宜蘭縣":"06宜蘭縣",
                             "苗栗縣":"07苗栗縣",
                             "臺中市":"08臺中市",
                             "彰化縣":"09彰化縣",
                             "南投縣":"10南投縣",
                             "雲林縣":"11雲林縣",
                             "嘉義市":"13嘉義市",
                             "嘉義縣":"12嘉義縣",
                             "臺南市":"14臺南市",
                             "高雄市":"15高雄市",
                             "屏東縣":"16屏東縣",
                             "澎湖縣":"19澎湖縣",
                             "花蓮縣":"17花蓮縣",
                             "臺東縣":"18臺東縣",
                             "金門縣":"20金門縣",
                             "連江縣":"21連江縣"})

In [ ]:
x=x.drop(["Unnamed: 0","Unnamed: 0.1"],axis=1)

In [19]:
x[x["鄉鎮"]=="員林鎮"]

,序號,觀察日期,縣市,鄉鎮,觀察地區,經度,緯度,科名,學名,中名,...,Village,Road_Name,distance,RdCode2,Town2,Village2,Road_Name2,distance2,臺灣分區,縣市(北中南東)
3807,3808,2015-07-25,彰化縣,員林鎮,NaN,120.631673,23.942591,Colubridae黃頷蛇科,Cyclophiops major,青蛇,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,中,09彰化縣
3808,3809,2015-07-25,彰化縣,員林鎮,NaN,120.631673,23.942591,Elapidae蝙蝠蛇科,Bungarus multicinctus multicinctus,雨傘節,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,中,09彰化縣
5335,5336,2015-06-12,彰化縣,員林鎮,NaN,120.622934,23.963348,Scincidae石龍子科,Plestiodon elegans,麗紋石龍子,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,中,09彰化縣
5452,5453,2015-06-09,彰化縣,員林鎮,NaN,120.617177,23.962058,Gekkonidae壁虎科,Hemidactylus frenatus,疣尾蝎虎,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,中,09彰化縣
5703,5704,2015-06-02,彰化縣,員林鎮,NaN,120.614729,23.961508,Ranidae赤蛙科,Hylarana latouchii,拉都希氏赤蛙,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,中,09彰化縣
5704,5705,2015-06-02,彰化縣,員林鎮,NaN,120.614729,23.961508,Agamidae飛蜥科,Japalura swinhonis,斯文豪氏攀蜥,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,中,09彰化縣
5705,5706,2015-06-02,彰化縣,員林鎮,NaN,120.614729,23.961508,Ranidae赤蛙科,Hylarana latouchii,拉都希氏赤蛙,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,中,09彰化縣
6307,6308,2015-05-13,彰化縣,員林鎮,NaN,120.614729,23.961508,Dicroglossidae叉舌蛙科,Fejervarya limnocharis,澤蛙,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,中,09彰化縣
6308,6309,2015-05-13,彰化縣,員林鎮,NaN,120.614729,23.961508,Dicroglossidae叉舌蛙科,Fejervarya limnocharis,澤蛙,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,中,09彰化縣
7133,7134,2015-04-07,彰化縣,員林鎮,NaN,120.620198,23.962946,Colubridae黃頷蛇科,Lycodon ruhstrati ruhstrati,白梅花蛇,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,中,09彰化縣


In [48]:
xx=x[x["縣市"]=="桃園市"]
set(xx["鄉鎮"])

{nan,
 '中壢區',
 '八德區',
 '大園區',
 '大溪區',
 '平鎮區',
 '復興區',
 '新屋區',
 '桃園區',
 '楊梅區',
 '樹林區',
 '蘆竹區',
 '觀音區',
 '龍潭區',
 '龜山區'}

In [47]:
x["鄉鎮"]=x["鄉鎮"].replace({"中壢市":"中壢區",
                             "八德市":"八德區",
                             "大園鄉":"大園區",
                             "大溪鎮":"大溪區",
                             "平鎮市":"平鎮區",
                             "復興鄉":"復興區",
                             "新屋鄉":"新屋區",
                             "桃園市":"桃園區",
                             "楊梅市":"楊梅區",
                             "蘆竹鄉":"蘆竹區",
                             "觀音鄉":"觀音區",
                             "龍潭鄉":"龍潭區",
                             "龜山鄉":"龜山區"})

In [52]:
x.loc[16641,"縣市"]="新北市"
x.loc[16641,"縣市(北中南東)"]="01新北市"
#x.loc[16641,]

In [57]:
x.to_excel("data(清理乾淨_增加節氣_增加溫溼度_增加道路資料B).xlsx")
x.to_excel("D:\\learning\\學士後大數據\\1072\\大數據專題\\data(清理乾淨_增加節氣_增加溫溼度_增加道路資料B).xlsx")

### 道路配適

In [23]:
x

,序號,觀察日期,縣市,鄉鎮,觀察地區,經度,緯度,科名,學名,中名,...,Village,Road_Name,distance,RdCode2,Town2,Village2,Road_Name2,distance2,臺灣分區,縣市(北中南東)
0,1,2015-12-31,臺中市,大肚區,NaN,120.572204,24.155384,Columbidae鳩鴿科,Streptopelia chinensis chinensis,珠頸斑鳩,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,中,08臺中市
1,8225,2014-12-23,臺東縣,蘭嶼鄉,椰油村288號,121.509903,22.056647,Strigidae鴟鴞科,Otus elegans botelensis,蘭嶼角鴞,...,椰油村,NaN,1.214749,Rd92304,蘭嶼鄉,椰油村,NaN,1.214749,東,18臺東縣
2,3,2015-12-31,臺北市,內湖區,NaN,121.572876,25.101981,Talpidae鼴鼠科,Mogera insularis insularis,台灣鼴鼠,...,金瑞里,大崙尾山步道,7.547951,Rd77566,內湖區,金瑞里,大崙尾山步道,7.547951,北,02臺北市
3,4,2015-12-31,臺北市,北投區,NaN,121.484387,25.153514,Colubridae黃頷蛇科,Oreocryptophis porphyraceus kawakamii,紅竹蛇,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,北,02臺北市
4,8259,2014-12-17,臺東縣,鹿野鄉,永樂路,121.126495,22.945644,Strigidae鴟鴞科,Otus,角鴞屬,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,東,18臺東縣
5,6,2015-12-31,嘉義市,西區,NaN,120.438899,23.479248,Soricidae尖鼠科,Suncus murinus,臭鼩,...,NaN,NaN,NaN,Rd2768,西區,後驛里,NaN,19.945795,南,13嘉義市
6,7,2015-12-31,桃園市,龜山區,NaN,121.318026,25.046119,Passeridae麻雀科,Passer montanus saturatus,麻雀,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,北,03桃園市
7,8307,2014-12-10,臺東縣,卑南鄉,花東公路,121.103622,22.881750,Strigidae鴟鴞科,Otus,角鴞屬,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,東,18臺東縣
8,8831,2014-11-02,臺中市,和平區,武陵農場,121.310131,24.352805,Strigidae鴟鴞科,Glaucidium brodiei,鵂鶹,...,平等里,NaN,10.397093,Rd68231,和平區,平等里,NaN,10.397093,中,08臺中市
9,10,2015-12-31,高雄市,茄萣區,NaN,120.212119,22.880803,Ardeidae鷺科,Nycticorax nycticorax nycticorax,夜鷺,...,NaN,NaN,NaN,Rd118506,茄萣區,崎漏里,NaN,35.843515,南,15高雄市


In [24]:
roaddata

,CODE,ROADNUM,ROADALIAS,COUNTY,TOWN,VILLAGE,ROAD_NAME,WIDTH,geometry
0,Rd0,None,None,南投縣,國姓鄉,國姓村,None,19.022789,"LINESTRING (120.8542707 24.0353081, 120.854340..."
1,Rd1,None,None,南投縣,鹿谷鄉,和雅村,None,19.022789,"LINESTRING (120.7661941 23.7127226, 120.766160..."
2,Rd2,None,None,南投縣,鹿谷鄉,和雅村,None,19.022789,"LINESTRING (120.7743284 23.716407, 120.7744386..."
3,Rd3,None,None,南投縣,鹿谷鄉,和雅村,None,19.022789,"LINESTRING (120.8031121 23.6989364, 120.802910..."
4,Rd4,None,None,南投縣,鹿谷鄉,和雅村,None,19.022789,"LINESTRING (120.7759961 23.6903228, 120.775816..."
5,Rd5,None,None,南投縣,鹿谷鄉,和雅村,None,19.022789,"LINESTRING (120.7755214 23.689161, 120.7757467..."
6,Rd6,None,None,南投縣,鹿谷鄉,和雅村,None,19.022789,"LINESTRING (120.7756704 23.6903686, 120.775700..."
7,Rd7,None,None,南投縣,鹿谷鄉,和雅村,None,19.022789,"LINESTRING (120.7761426 23.6903155, 120.776165..."
8,Rd8,None,None,南投縣,鹿谷鄉,和雅村,None,19.022789,"LINESTRING (120.7683261 23.6832372, 120.768405..."
9,Rd9,None,None,南投縣,鹿谷鄉,彰雅村,None,19.022789,"LINESTRING (120.770731 23.7527633, 120.7708129..."


In [27]:
x["Road_num2"]=[""]*len(x)
for i in range(len(x)):
    for j in range(len(roaddata)):
        if x.loc[i,"RdCode2"]==roaddata.loc[j,"CODE"]:
            x.loc[i,"Road_num2"]=roaddata.loc[j,"ROADNUM"]
            break

In [32]:
x

,序號,觀察日期,縣市,鄉鎮,觀察地區,經度,緯度,科名,學名,中名,...,Road_Name,distance,RdCode2,Town2,Village2,Road_Name2,distance2,臺灣分區,縣市(北中南東),Road_num2
0,1,2015-12-31,臺中市,大肚區,NaN,120.572204,24.155384,Columbidae鳩鴿科,Streptopelia chinensis chinensis,珠頸斑鳩,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,中,08臺中市,
1,8225,2014-12-23,臺東縣,蘭嶼鄉,椰油村288號,121.509903,22.056647,Strigidae鴟鴞科,Otus elegans botelensis,蘭嶼角鴞,...,NaN,1.214749,Rd92304,蘭嶼鄉,椰油村,NaN,1.214749,東,18臺東縣,None
2,3,2015-12-31,臺北市,內湖區,NaN,121.572876,25.101981,Talpidae鼴鼠科,Mogera insularis insularis,台灣鼴鼠,...,大崙尾山步道,7.547951,Rd77566,內湖區,金瑞里,大崙尾山步道,7.547951,北,02臺北市,None
3,4,2015-12-31,臺北市,北投區,NaN,121.484387,25.153514,Colubridae黃頷蛇科,Oreocryptophis porphyraceus kawakamii,紅竹蛇,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,北,02臺北市,
4,8259,2014-12-17,臺東縣,鹿野鄉,永樂路,121.126495,22.945644,Strigidae鴟鴞科,Otus,角鴞屬,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,東,18臺東縣,
5,6,2015-12-31,嘉義市,西區,NaN,120.438899,23.479248,Soricidae尖鼠科,Suncus murinus,臭鼩,...,NaN,NaN,Rd2768,西區,後驛里,NaN,19.945795,南,13嘉義市,None
6,7,2015-12-31,桃園市,龜山區,NaN,121.318026,25.046119,Passeridae麻雀科,Passer montanus saturatus,麻雀,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,北,03桃園市,
7,8307,2014-12-10,臺東縣,卑南鄉,花東公路,121.103622,22.881750,Strigidae鴟鴞科,Otus,角鴞屬,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,東,18臺東縣,
8,8831,2014-11-02,臺中市,和平區,武陵農場,121.310131,24.352805,Strigidae鴟鴞科,Glaucidium brodiei,鵂鶹,...,NaN,10.397093,Rd68231,和平區,平等里,NaN,10.397093,中,08臺中市,None
9,10,2015-12-31,高雄市,茄萣區,NaN,120.212119,22.880803,Ardeidae鷺科,Nycticorax nycticorax nycticorax,夜鷺,...,NaN,NaN,Rd118506,茄萣區,崎漏里,NaN,35.843515,南,15高雄市,None


In [39]:
x["國省道號或路名"]=[""]*len(x)
for i in range(len(x)):
    if pd.isnull(x.loc[i,"Road_num2"]):
        x.loc[i,"Road_num2"]=""
    if pd.isnull(x.loc[i,"Road_Name2"]):
        x.loc[i,"Road_Name2"]=""
    x.loc[i,"國省道號或路名"]=x.loc[i,"Road_num2"]+"  "+x.loc[i,"Road_Name2"]

In [41]:
x.to_excel("data(清理乾淨_增加節氣_增加溫溼度_增加道路資料B).xlsx")
x.to_excel("D:\\learning\\學士後大數據\\1072\\大數據專題\\data(清理乾淨_增加節氣_增加溫溼度_增加道路資料B).xlsx")

In [40]:
x

,序號,觀察日期,縣市,鄉鎮,觀察地區,經度,緯度,科名,學名,中名,...,distance,RdCode2,Town2,Village2,Road_Name2,distance2,臺灣分區,縣市(北中南東),Road_num2,國省道號或路名
0,1,2015-12-31,臺中市,大肚區,NaN,120.572204,24.155384,Columbidae鳩鴿科,Streptopelia chinensis chinensis,珠頸斑鳩,...,NaN,NaN,NaN,NaN,,NaN,中,08臺中市,,
1,8225,2014-12-23,臺東縣,蘭嶼鄉,椰油村288號,121.509903,22.056647,Strigidae鴟鴞科,Otus elegans botelensis,蘭嶼角鴞,...,1.214749,Rd92304,蘭嶼鄉,椰油村,,1.214749,東,18臺東縣,,
2,3,2015-12-31,臺北市,內湖區,NaN,121.572876,25.101981,Talpidae鼴鼠科,Mogera insularis insularis,台灣鼴鼠,...,7.547951,Rd77566,內湖區,金瑞里,大崙尾山步道,7.547951,北,02臺北市,,大崙尾山步道
3,4,2015-12-31,臺北市,北投區,NaN,121.484387,25.153514,Colubridae黃頷蛇科,Oreocryptophis porphyraceus kawakamii,紅竹蛇,...,NaN,NaN,NaN,NaN,,NaN,北,02臺北市,,
4,8259,2014-12-17,臺東縣,鹿野鄉,永樂路,121.126495,22.945644,Strigidae鴟鴞科,Otus,角鴞屬,...,NaN,NaN,NaN,NaN,,NaN,東,18臺東縣,,
5,6,2015-12-31,嘉義市,西區,NaN,120.438899,23.479248,Soricidae尖鼠科,Suncus murinus,臭鼩,...,NaN,Rd2768,西區,後驛里,,19.945795,南,13嘉義市,,
6,7,2015-12-31,桃園市,龜山區,NaN,121.318026,25.046119,Passeridae麻雀科,Passer montanus saturatus,麻雀,...,NaN,NaN,NaN,NaN,,NaN,北,03桃園市,,
7,8307,2014-12-10,臺東縣,卑南鄉,花東公路,121.103622,22.881750,Strigidae鴟鴞科,Otus,角鴞屬,...,NaN,NaN,NaN,NaN,,NaN,東,18臺東縣,,
8,8831,2014-11-02,臺中市,和平區,武陵農場,121.310131,24.352805,Strigidae鴟鴞科,Glaucidium brodiei,鵂鶹,...,10.397093,Rd68231,和平區,平等里,,10.397093,中,08臺中市,,
9,10,2015-12-31,高雄市,茄萣區,NaN,120.212119,22.880803,Ardeidae鷺科,Nycticorax nycticorax nycticorax,夜鷺,...,NaN,Rd118506,茄萣區,崎漏里,,35.843515,南,15高雄市,,
